# Understanding Decorators in Python

In Python, decorators are tools that allow you to modify the behavior of a function or class. They are like wrappers that add functionality to your code in a clean and reusable way.

**Here’s an example:** creating a decorator to calculate the execution time of a function.

In [1]:
import time

def execution_time_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()  # Record the start time with perf_counter
        result = func(*args, **kwargs)  # Call the original function
        end_time = time.perf_counter()  # Record the end time with perf_counter
        execution_time = end_time - start_time  # Calculate execution time
        print(f"Execution time of {func.__name__}: {execution_time:.4f} seconds")
        return result  # Return the result of the original function
    return wrapper

# Using the decorator
@execution_time_decorator
def example_function(seconds):
    time.sleep(seconds)

# Test the function
example_function(2)

Execution time of example_function: 2.0004 seconds


## Explanation:

1. **Define the Decorator:** `execution_time_decorator` takes a function func as its argument.
2. **Wrapper Function:** Inside, `wrapper` is defined to wrap the original function. It records the start and end times, calculates the execution time, and prints it.
3. **Using the Decorator:** The `@execution_time_decorator` syntax is a shorthand to apply the decorator to example_function.
When you call `example_function(2)`, it will sleep for 2 seconds and then print the execution time.

## Why Use Decorators?

- **Code Reusability:** Write once, use multiple times.
- **Separation of Concerns:** Keep core logic and additional functionality separate.
- **Readability:** Cleaner and more readable code.

# Here are several practical use cases where decorators are commonly employed:

## 1. Logging

Decorators can be used to log function calls, including the arguments passed and the result returned.

In [2]:
import logging
from functools import wraps

logging.basicConfig(level=logging.INFO)

def log_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        logging.info(f"Calling {func.__name__} with args {args} and kwargs {kwargs}")
        result = func(*args, **kwargs)
        logging.info(f"{func.__name__} returned {result}")
        return result
    return wrapper

@log_decorator
def add(a, b):
    return a + b

add(2, 3)

INFO:root:Calling add with args (2, 3) and kwargs {}
INFO:root:add returned 5


5

## 2. Access Control

Decorators can enforce access control by checking user permissions before allowing access to a function.

In [3]:
from functools import wraps

def requires_permission(permission):
    def decorator(func):
        @wraps(func)
        def wrapper(user, *args, **kwargs):
            if not user.has_permission(permission):
                raise PermissionError("You do not have the required permission")
            return func(user, *args, **kwargs)
        return wrapper
    return decorator

class User:
    def __init__(self, permissions):
        self.permissions = permissions
    
    def has_permission(self, permission):
        return permission in self.permissions

@requires_permission('admin')
def delete_user(user, username):
    print(f"User {username} deleted")

admin_user = User(['admin'])
regular_user = User([])

delete_user(admin_user, 'john_doe')  # Allowed
# delete_user(regular_user, 'john_doe')  # Raises PermissionError

User john_doe deleted


## 3. Caching

Decorators can cache function results to avoid redundant computations, improving performance.

In [4]:
from functools import wraps

def cache_decorator(func):
    cache = {}
    
    @wraps(func)
    def wrapper(*args):
        if args in cache:
            return cache[args]
        result = func(*args)
        cache[args] = result
        return result
    
    return wrapper

@cache_decorator
def compute_square(n):
    return n * n

print(compute_square(4))  # Computes and caches result
print(compute_square(4))  # Returns cached result

16
16


## 4. Input Validation

Decorators can validate the inputs to a function, ensuring they meet certain criteria before execution.

In [5]:
from functools import wraps

def validate_non_negative(func):
    @wraps(func)
    def wrapper(x, y):
        if x < 0 or y < 0:
            raise ValueError("Arguments must be non-negative")
        return func(x, y)
    return wrapper

@validate_non_negative
def add(x, y):
    return x + y

print(add(2, 3))  # Valid
# print(add(-1, 2))  # Raises ValueError

5


## 5. Memoization

A specific form of caching that remembers previous function arguments and results.

In [6]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(fibonacci(10))  # Efficiently computes with memoization

55


## 6. Rate Limiting

Decorators can limit the number of times a function can be called within a specific timeframe.

In [7]:
import time
from functools import wraps

def rate_limit(max_calls, period):
    calls = []

    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            current_time = time.perf_counter()
            calls[:] = [call for call in calls if current_time - call < period]
            if len(calls) >= max_calls:
                raise Exception("Rate limit exceeded")
            calls.append(current_time)
            return func(*args, **kwargs)
        return wrapper
    return decorator

@rate_limit(3, 10)  # Allow max 3 calls per 10 seconds
def limited_function():
    print("Function called")

limited_function()
limited_function()
limited_function()
# limited_function()  # Raises Exception: Rate limit exceeded

Function called
Function called
Function called


# Conclusion

Decorators in Python offer a flexible and powerful method for extending and modifying the behavior of functions and methods without altering their code. They are extensively utilized in real-world applications for tasks such as logging, access control, caching, input validation, performance measurement, and more.